# WaveformParam

This notebook provides examples on how to use siriuspy.ramp subpackage in order to manipulate booster ramps and storage ring migration power supply waveforms. 

In [ ]:
from lnls import notebook
notebook.turn_code_on_off_html()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from siriuspy.ramp.waveform import WaveformParam

## Bare constructor

In [ ]:
# With no arguments the constructor creates a default waveform

w = WaveformParam()

%matplotlib inline
plt.plot(w)
plt.xlabel('Index'); plt.ylabel('Waveform [a.u.]')
plt.grid('on')
plt.show()

# By default the value 1.0 should correspond to the beam ejection point, 
# whereas the value 1.0/20 (20 = E_eje/E_inj) should correspond to the injection point:

idx_inj = 160
idx_eje = 2400
print('value at maximum         : {:.8f}'.format(max(w)))
print('value at injection point : {:.8f} (idx = {})'.format(w[idx_inj],idx_inj))
print('value at ejection point  : {:.8f} (idx = {})'.format(w[idx_eje],idx_eje))

## Parametrization

The parametrized component of the waveform is composed of 9 disctinct regions. There are therefore 8 indices that define the boundary limits of each region. The first and last regions, for the default waveform, have 1-point widths. In each regian the default waveform is a polynominal curve of varying order, with continous current and first derivative.

In [ ]:
w = WaveformParam()
bi = w.boundary_indices
print('Regions internal limits: {}'.format(bi))
print('')
print('Region 0 (linear), 0  <= i < {}'.format(bi[0]))
print('Region 1 (cubic) , {0} <= i < {1}'.format(bi[0], bi[1]))
print('Region 2 (linear), {0} <= i < {1} (ramp up)'.format(bi[1], bi[2]))
print('Region 3 (cubic),  {0} <= i < {1}'.format(bi[2], bi[3]))
print('Region 4 (linear), {0} <= i < {1} (plateau)'.format(bi[3], bi[4]))
print('Region 5 (cubic),  {0} <= i < {1}'.format(bi[4], bi[5]))
print('Region 6 (linear), {0} <= i < {1} (ramp down)'.format(bi[5], bi[6]))
print('Region 7 (cubic),  {0} <= i < {1}'.format(bi[6], bi[7]))
print('Region 8 (linear), {0} <= i < {1}'.format(bi[7], len(w)))

w_min, w_max = min(w), max(w)
%matplotlib inline
plt.plot(w)
plt.plot([bi[0],bi[0]],[-w_min,w_max], 'k--')
plt.plot([bi[1],bi[1]],[-w_min,w_max], 'k--')
plt.plot([bi[2],bi[2]],[-w_min,w_max], 'k--')
plt.plot([bi[3],bi[3]],[-w_min,w_max], 'k--')
plt.plot([bi[4],bi[4]],[-w_min,w_max], 'k--')
plt.plot([bi[5],bi[5]],[-w_min,w_max], 'k--')
plt.plot([bi[6],bi[6]],[-w_min,w_max], 'k--')
plt.plot([bi[7],bi[7]],[-w_min,w_max], 'k--')
plt.show()

In the plot above limit lines for Region0-Region1 and Region7-Region8 are not discernable since the first and the last regions are only 1 point wide.

Waveform values at these 8 limit indices and at the waveform extremities can also be accessed using class interface methods. Method *start_value* impose the same value at the waveform start and at the first region boundary. Likewise, *plateau_value* impose the same value at start and stop indices of the plateau region:

In [ ]:
print('Value at i=0 and at the first region boundary is given by property "start_value"      : {:.4f}'.format(w.start_value))
print('Value at the start of the ramp up region is given by property "rampup_start_value"    : {:.4f}'.format(w.rampup_start_value))
print('Value at the end of the ramp up region is given by property "rampup_stop_value"       : {:.4f}'.format(w.rampup_stop_value))
print('Value at the plateau region is given by property "plateau_value"                      : {:.4f}'.format(w.plateau_value))
print('Value at the start of the ramp down region is given by property "rampdown_start_value": {:.4f}'.format(w.rampdown_start_value))
print('Value at the end of the ramp down region is given by property "rampdown_stop_value"   : {:.4f}'.format(w.rampdown_stop_value))
print('Value at i=3999 and at the end of the waveform is given by property "stop_value"      : {:.4f}'.format(w.stop_value))

## Changing RampUp and RampDown

There are utility methods that allow changing both linear ramp up and ramp down without waveform 'overshoots' and guaranteeing continuities. For that, the first region boundary and *plateau_start_index* are reset for ramp up changes, as collateral effect. Likewise, *plateau_stop_index* and the last region boundary are reset for ramp down changes:

In [ ]:
w = WaveformParam()
w0 = [0*0.1 + v for v in w.waveform]

w.rampup_change(start=w.rampup_start_index, stop=w.rampup_stop_index-500, start_value=w.rampup_start_value, stop_value=w.rampup_stop_value)
w1 = [1*0.1 + v for v in w.waveform]

w.rampdown_change(stop=w.rampdown_stop_index-500)
w2 = [2*0.1 + v for v in w.waveform]

%matplotlib inline
plt.plot(w0, label='original')
plt.plot(w1, label='change ramp up')
plt.plot(w2, label='change ramp up and down')
plt.legend()
plt.xlabel('Index'); plt.ylabel('Waveform [a.u.]')
plt.grid('on')
plt.show()